# Import necessary libraries

In [ ]:
import os
import numpy as np
from imutils import paths
from keras.models import Model
from keras.layers import AveragePooling2D,Dense,Dropout,Flatten
from keras.applications import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img,img_to_array
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer

# Training of the model

In [ ]:
base_dir=r"C:/Users/Tuhin Roy/Downloads/dataset"
categories=["with_mask","without_mask"]
data=[]
labels=[]
for cat in categories:
    path=os.path.join(base_dir,cat)
    for img in os.listdir(path):
        img_path=os.path.join(path,img)
        images=load_img(img_path,target_size=(224,224))
        images=img_to_array(images)
        images=preprocess_input(images)
        data.append(images)
        labels.append(cat)

lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

data=np.array(data,dtype="float32")
labels=np.array(labels)

X_train,X_test,y_train,y_test=train_test_split(data,labels,stratify=labels,test_size=0.2,random_state=42)

baseModel=MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')
headModel=baseModel.output
headModel=AveragePooling2D(pool_size=(7,7))(headModel)
headModel=Flatten(name="flatten")(headModel)
headModel=Dense(128,activation="relu")(headModel)
headModel=Dropout(0.5)(headModel)
headModel=Dense(2,activation="softmax")(headModel)
model=Model(inputs=baseModel.input,outputs=headModel)
for layer in baseModel.layers:
    layer.trainable=False
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20)

model.save("mask_recog.h5")

# Load model

In [ ]:
import cv2
from keras.models import load_model
model=load_model("mask_detection.h5")

# Image Face Mask Detection

In [ ]:
faceCascade=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
labels_dict={1:"No Mask",0:"Mask"}
color_dict={0:(0,255,0),1:(0,0,255)}
frame=cv2.imread("test2.jpg")
rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
faces=faceCascade.detectMultiScale(rgb,1.1,3)
for (x,y,w,h) in faces:
    face_img=frame[y:y+h,x:x+w]
    images=cv2.resize(face_img,(224,224))
    images=img_to_array(images)
    images=preprocess_input(images)
    images=np.array([images],dtype="float32")
    pred=model.predict(images)
    label=np.argmax(pred,axis=1)[0]
    percen=str(" : ")+str((np.max(pred)*100).round(2))+str("%")
    cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
    if (label==1):
        cv2.putText(frame,labels_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
        cv2.putText(frame,percen,(x+70,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
    else:
        cv2.putText(frame,labels_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
        cv2.putText(frame,percen,(x+40,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
cv2.imshow("Face Mask Detection",frame)
cv2.waitKey()
cv2.destroyAllWindows()

# Real-time Face Mask Detection

In [ ]:
cascade=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt2.xml')
labels_dict={0:"MASK",1:"NO MASK"}
color_dict={0:(0,255,0),1:(0,0,255)}
cap=cv2.VideoCapture(0)
while(True):
    _,img=cap.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    faces=cascade.detectMultiScale(gray,1.1,3)
    for (x,y,w,h) in faces:
        face_img=img[y:y+w,x:x+w]
        image=cv2.resize(face_img,(224,224))
        image=img_to_array(image)
        image=preprocess_input(image)
        image=np.array([image],dtype="float32")
        pred=model.predict(image)
        sed=str(" : ")+str((np.max(pred)*100).round(2))+str("%")
        label=np.argmax(pred,axis=1)[0]
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        if (label==1):
            cv2.putText(img,labels_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
            cv2.putText(img,sed,(x+70,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2)
        else:
            cv2.putText(img,labels_dict[label],(x,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
            cv2.putText(img,sed,(x+40,y-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2)
    cv2.imshow("Face Mask Detection",img)
    if cv2.waitKey(1) & 0XFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()